In [61]:
import berreman4x4 as bm

In [62]:
MM = bm.SpectraRay.read_mmatrix('Wafer_MM_70.txt').loc[210:820]

# Swap absorptive indices to conform to n + ik sign convention
MM.M43 *= -1
MM.M34 *= -1

In [63]:
params = bm.ParamsHist()
params.add('SiO2_n0', value=1.452, min=-100, max=100, vary=True)
params.add('SiO2_n1', value=36.0, min=-40000, max=40000, vary=True)
params.add('SiO2_n2', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_k0', value=0, min=-100, max=100, vary=True)
params.add('SiO2_k1', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_k2', value=0, min=-40000, max=40000, vary=True)
params.add('SiO2_d', value=120, min=0, max=40000, vary=True)

In [64]:
@bm.fit_mueller_matrix(MM, params, display_single=False)
def model(lbda, params):
    sr = bm.SpectraRay('./')
    Si = bm.IsotropicMaterial(sr.loadDispersionTable('Si_Aspnes.mat'))

    SiO2 = bm.IsotropicMaterial(bm.DispersionCauchy(params['SiO2_n0'], 
                                                    params['SiO2_n1'], 
                                                    params['SiO2_n2'], 
                                                    params['SiO2_k0'], 
                                                    params['SiO2_k1'], 
                                                    params['SiO2_k2']))
    
    Layer = [bm.Layer(SiO2, params['SiO2_d'])]
    
    return bm.Structure(bm.AIR, Layer, Si).evaluate(lbda, 70, solver=bm.Solver4x4, propagator=bm.PropagatorExpmTorch())

In [65]:
model.fit()

In [66]:
model.plot()

FigureWidget({
    'data': [{'line': {'color': '#636EFA', 'dash': 'solid'},
              'name': 'M11 ',
    …